In [198]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader,Dataset
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
from scipy.stats import wasserstein_distance

# Data Preprocessing

In [199]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

class DataPreprocessing():
    def __init__(self,data,weigths):
        self.CLIP_MIN = 13 # const values to indicate max and min of clipping
        self.CLIP_MAX = 90
        self.SCALE_PARAM =3.5
        self.dataframe = data 
        self.weights = weigths
        self.unscaled_y_train = 0
        self.unscaled_y_test = 0
        self.train_mean = 0 # mean value of training data :163 columns
        self.group_count  = 0
        
    def UpdateWeights(self):
        """Add weights for the last three columns of the dataframe"""
        self.weights = pd.concat([self.weights,pd.Series([1,1,1])],ignore_index=True)
        
    def CorrectOutliers(self):
        """Handle outliers existing in some columns"""
        
        self.dataframe['A'] = self.dataframe['A'].clip(self.CLIP_MIN,self.CLIP_MAX)
        
    def EncodeStrings(self):
        """Encode string values of column C to numeric values in range [-1,1]"""
        
        encoding = np.linspace(-1,1,160)
        mappings = dict(zip(self.dataframe['C'].unique(), encoding))
        self.dataframe['C'] = self.dataframe['C'].map(mappings)
      
    def MultiplyWeights(self):
        """Multiply columns with their weights"""
    
        self.dataframe = self.dataframe.mul(np.array(self.weights).reshape(1,-1), axis = 1).copy()
  
    def RandomChoice(self):
        """Choose random 30 column indices and concatenate last three column indices"""
    
        return np.concatenate((np.random.choice([i for i in range(163)],30, replace=False),[163,164,165]))
    
        
    def SplitData(self,train_size=0.8):
        """Split unscaled data to train, test, and save unscaled values of train and test data 
        in purpose of using them during Ai model testing step"""
       
        train_data, test_data=  train_test_split(self.dataframe,train_size=train_size) ##166 columns
        self.unscaled_y_train = train_data.iloc[:,:-3].copy() 
        self.unscaled_y_test = test_data.iloc[:,:-3].copy()  
        return train_data,test_data
    
    def SaveTrainMean(self,train_data):
        self.train_mean = (train_data.iloc[:,:-3].mean()).values
        
    def NormalizeData(self,data):
        """Normalize data to [-1,1]"""
        data = data.copy()
        if data.shape[1]== self.dataframe.shape[1]:  # normlization for input data(containing 166 features)
            data.loc[:,'G'].replace({0:1, 1:-1, 2:0, 3:1}, inplace=True)
            data.loc[:,'A']  = 2*((data.loc[:,'A']-self.CLIP_MIN)/(self.CLIP_MAX-self.CLIP_MIN))-1
            data.iloc[:,:-3] = (data.iloc[:,:-3] - self.train_mean) / self.SCALE_PARAM
        else:
            data =(data - self.train_mean) / self.SCALE_PARAM #normalization for output  data (163 features)
        return data
 
    def GetAiInput(self,scaled_train_data,scaled_test_data,indices):
      #  Take 30, take 166 mean vector, put 30 into 163 , put into 164-166,
       # then we have here updated mean vector (not all mean anymore)"""
        
        col_indices = self.dataframe.columns[indices]
        means = scaled_train_data.mean() # mean of 166 cols, to create numpy array filled with that means
        new_train = np.zeros(scaled_train_data.shape)
        new_test = np.zeros(scaled_test_data.shape)
        new_test[:,indices] = scaled_test_data[col_indices] # fill 33 columns of mean vector of actual values
        
        new_train[:,indices] = scaled_train_data[col_indices]
        
        #get uncaled input and output  normalized train and test data(type=dataframe)
        self.x_train = pd.DataFrame(new_train, columns = self.dataframe.columns)
        self.x_test = pd.DataFrame(new_test,columns = self.dataframe.columns)
        self.y_train = scaled_train_data.iloc[:,:-3].copy() 
        self.y_test = scaled_test_data.iloc[:,:-3].copy()  #tarrget output (163 features)
        return self.x_train,self.x_test,self.y_train,self.y_test
    
        
    def GroupNumber(self):
        """save size of each group
        return indexing for spliting data with groups in training and testing of Ai model"""
        first_letters = [i[0]for i in list(self.dataframe.iloc[:,:-3].columns)]
        self.group_count = [first_letters.count(el) for el in np.unique(first_letters)]
        return [sum(self.group_count[0:i]) for i in range (1,len(self.group_count))]

    def CorrectData(self):
        self.UpdateWeights()
        self.CorrectOutliers()
        self.EncodeStrings()
        self.MultiplyWeights()
        

In [200]:
dataframe =pd.read_csv("./intern_data/data.csv") 
weights = pd.read_csv("./intern_data/weights.csv",header=None)

In [201]:
preprocess= DataPreprocessing(dataframe,weights)

In [202]:
indices = preprocess.RandomChoice()

In [203]:
preprocess.CorrectData()

In [204]:
train_data, test_data = preprocess.SplitData()
train_data

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,P4,P5,P6,P7,P8,P9,P10,A,G,C
44638,3,4,4,4,2,2,4,-2,-2,-4,...,4,4,3,3,-2,-4,-3,19,2,-0.823899
40002,5,5,5,5,4,4,5,-2,-1,-1,...,2,1,5,3,-3,-4,-5,39,2,-1.000000
26361,4,4,4,3,5,4,4,-3,-3,-1,...,2,2,4,2,-4,-3,-4,19,2,-1.000000
23496,4,4,5,4,4,4,4,-2,-1,-3,...,2,3,4,2,-4,-5,-4,23,2,-1.000000
18111,2,3,4,3,4,3,2,-4,-3,-3,...,4,2,3,3,-3,-4,-4,18,2,-0.597484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48401,4,4,4,4,4,3,5,-3,-2,-1,...,3,2,2,1,-3,-4,-3,25,2,-1.000000
46617,5,3,5,3,4,4,3,-3,-2,-3,...,5,3,4,4,-1,-4,-2,16,2,-1.000000
10490,4,5,4,4,5,4,5,-3,-2,-2,...,2,2,2,4,-4,-4,-4,59,1,-0.886792
14869,5,5,4,4,5,5,4,-3,-1,-1,...,4,2,3,2,-3,-3,-4,28,1,-1.000000


In [205]:
scaled_train = preprocess.NormalizeData(train_data)

In [206]:
scaled_train

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,P4,P5,P6,P7,P8,P9,P10,A,G,C
44638,0.857143,1.142857,1.142857,1.142857,0.571429,0.571429,1.142857,-0.571429,-0.571429,-1.142857,...,1.142857,1.142857,0.857143,0.857143,-0.571429,-1.142857,-0.857143,-0.844156,0,-0.823899
40002,1.428571,1.428571,1.428571,1.428571,1.142857,1.142857,1.428571,-0.571429,-0.285714,-0.285714,...,0.571429,0.285714,1.428571,0.857143,-0.857143,-1.142857,-1.428571,-0.324675,0,-1.000000
26361,1.142857,1.142857,1.142857,0.857143,1.428571,1.142857,1.142857,-0.857143,-0.857143,-0.285714,...,0.571429,0.571429,1.142857,0.571429,-1.142857,-0.857143,-1.142857,-0.844156,0,-1.000000
23496,1.142857,1.142857,1.428571,1.142857,1.142857,1.142857,1.142857,-0.571429,-0.285714,-0.857143,...,0.571429,0.857143,1.142857,0.571429,-1.142857,-1.428571,-1.142857,-0.740260,0,-1.000000
18111,0.571429,0.857143,1.142857,0.857143,1.142857,0.857143,0.571429,-1.142857,-0.857143,-0.857143,...,1.142857,0.571429,0.857143,0.857143,-0.857143,-1.142857,-1.142857,-0.870130,0,-0.597484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48401,1.142857,1.142857,1.142857,1.142857,1.142857,0.857143,1.428571,-0.857143,-0.571429,-0.285714,...,0.857143,0.571429,0.571429,0.285714,-0.857143,-1.142857,-0.857143,-0.688312,0,-1.000000
46617,1.428571,0.857143,1.428571,0.857143,1.142857,1.142857,0.857143,-0.857143,-0.571429,-0.857143,...,1.428571,0.857143,1.142857,1.142857,-0.285714,-1.142857,-0.571429,-0.922078,0,-1.000000
10490,1.142857,1.428571,1.142857,1.142857,1.428571,1.142857,1.428571,-0.857143,-0.571429,-0.571429,...,0.571429,0.571429,0.571429,1.142857,-1.142857,-1.142857,-1.142857,0.194805,-1,-0.886792
14869,1.428571,1.428571,1.142857,1.142857,1.428571,1.428571,1.142857,-0.857143,-0.285714,-0.285714,...,1.142857,0.571429,0.857143,0.571429,-0.857143,-0.857143,-1.142857,-0.610390,-1,-1.000000


In [207]:
scaled_test = preprocess.NormalizeData(test_data)

# Separate Ai input and labels

In [208]:
x_train, x_test, y_train, y_test = preprocess.GetAiInput(scaled_train,scaled_test,indices)

# Denoising Autoencoder based architecture

# Dataloader

In [134]:
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
import torch

class MyDataset(Dataset):
    def __init__(self, X_train, Y_train):
        x = X_train.values
        y = Y_train.values
        self.x_train= torch.from_numpy(x).float()
        self.y_train= torch.from_numpy(y).float()
        
    def __len__(self):
        return len(self.y_train)
   
    def __getitem__(self,idx):
        return self.x_train[idx].float(), self.y_train[idx].float()
    

# Model training


In [148]:
   def TrainNN(train_loader,num_epochs,loss_function):
        for epoch in range(num_epochs):
            acc=0
            print("Epoch: " + str(epoch))
            epoch_loss = 0
            for i, (data, targets) in enumerate(train_loader):
                predictions = model(data)
                loss = loss_function(predictions, targets)
                epoch_loss += loss
                #Backward
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            print("loss: ",epoch_loss/i)

# Model Testing

In [224]:
#Accuarcy computed for whole dataset
def CheckAccuracy(model,input_scaled,target_unscaled):
    input_scaled = torch.from_numpy(input_scaled.values).float().unsqueeze(1)
    target_unscaled = torch.from_numpy(target_unscaled.values).float()
    with torch.no_grad():
        pred_scaled = model(input_scaled)
    prediction_float = (pred_scaled * 3.5) + preprocess.train_mean
    prediction_int = torch.round(prediction_float)
    prediction_int = torch.reshape(prediction_int,(prediction_int.shape[0],prediction_int.shape[-1]))
    equality_matrix = (prediction_int==target_unscaled)
    num_correct = equality_matrix.sum()
    num_samples = (equality_matrix.shape[0]*equality_matrix.shape[-1])
    print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples):.5f}') 


# With dense layers

In [150]:
class DAE(torch.nn.Module):
    def __init__(self):
        super(DAE,self).__init__()
        self.encoder = torch.nn.Sequential (
        torch.nn.Linear(166, 128),
        torch.nn.ReLU(),
        torch.nn.Linear(128, 64),
        torch.nn.ReLU(),
        torch.nn.Linear(64, 36),
        torch.nn.ReLU(),
        torch.nn.Linear(36, 18),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(18, 9))
        self.decoder = torch.nn.Sequential (
        torch.nn.Linear(9, 18),
        torch.nn.ReLU(),
        torch.nn.Linear(18, 36),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(36, 64),
        torch.nn.ReLU(),
        torch.nn.Linear(64, 128),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(128, 163))
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


In [151]:
model = DAE()
loss_function = nn.MSELoss()
#loss_function = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr = learning_rate) 
batch_size = 30
num_epochs = 10

In [152]:
TrainNN(train_loader,num_epochs,loss_function)


Epoch: 0
loss:  tensor(0.1046, grad_fn=<DivBackward0>)
Epoch: 1
loss:  tensor(0.0864, grad_fn=<DivBackward0>)
Epoch: 2
loss:  tensor(0.0851, grad_fn=<DivBackward0>)
Epoch: 3
loss:  tensor(0.0836, grad_fn=<DivBackward0>)
Epoch: 4
loss:  tensor(0.0789, grad_fn=<DivBackward0>)
Epoch: 5
loss:  tensor(0.0774, grad_fn=<DivBackward0>)
Epoch: 6
loss:  tensor(0.0760, grad_fn=<DivBackward0>)
Epoch: 7
loss:  tensor(0.0752, grad_fn=<DivBackward0>)
Epoch: 8
loss:  tensor(0.0750, grad_fn=<DivBackward0>)
Epoch: 9
loss:  tensor(0.0748, grad_fn=<DivBackward0>)


In [153]:
CheckAccuracy(model,x_train,preprocess.unscaled_y_train)

Got 708632 / 6410301 with accuracy 0.11055


In [154]:
CheckAccuracy(model,x_test,preprocess.unscaled_y_test)

Got 178795 / 1602616 with accuracy 0.11156


# residual network for CNNI inputation

In [162]:
#Convolution 1D
class ResidualNet(nn.Module): 
    def __init__(self):  
        super(ResidualNet, self).__init__()

        self.conv1 = torch.nn.Conv1d(in_channels=1, out_channels=1, kernel_size=10, stride=1,
                        padding=0, dilation=1, groups=1, bias=True, 
                        padding_mode='replicate', device=None, dtype=None)
        
        self.max_pool1 = torch.nn.AvgPool1d(kernel_size=3, stride=None, padding=0, ceil_mode=False, count_include_pad=True)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(in_channels=1, out_channels=10, kernel_size=3, stride=1,
                        padding=0, dilation=1, groups=1, bias=True, 
                        padding_mode='reflect', device=None, dtype=None)
        
        self.conv3 = nn.Conv1d(in_channels=10, out_channels=20, kernel_size=3, stride=1,
                        padding=0, dilation=1, groups=1, bias=True, 
                        padding_mode='reflect', device=None, dtype=None)
        
        self.conv4 = nn.Conv1d(in_channels=20, out_channels=1, kernel_size=5, stride=1,
                        padding=0, dilation=1, groups=1, bias=True, 
                        padding_mode='zeros', device=None, dtype=None)
        self.conv5 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3, stride=1,
                        padding=0, dilation=1, groups=1, bias=True, 
                        padding_mode='zeros', device=None, dtype=None)
        
        self.conv6 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=5, stride=1,
                        padding=0, dilation=1, groups=1, bias=True, 
                        padding_mode='zeros', device=None, dtype=None)
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
        self.dense1 = nn.Linear(10,163)
        self.dense2 = nn.Linear(100,163)
        self.dense3 = nn.Linear(163,163)
               
    def forward(self,x):
        res = x
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool1(x)
        x= self.conv2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x= self.max_pool1(x)
        x= self.relu(x)
        x = self.conv4(x)
        x= self.relu(x)
        x = self.conv5(x)
        x = self.relu(x)
        x = self.dense1(x)
        x  = x + res[:,:,:163] 
        x = self.dense3(x)
        return x

In [163]:
model = ResidualNet()
loss_function = nn.MSELoss()
learning_rate = 0.001
#optimizer = optim.Adam(model.parameters(), lr = learning_rate) 
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9) 
batch_size = 30
num_epochs = 10

In [164]:
   def TrainRCNN(model,train_loader, num_epochs, loss_function):

        for epoch in range(num_epochs):
            acc=0
            print("Epoch: " + str(epoch))
            epoch_loss = 0
            for i, (data, targets) in enumerate(train_loader):
                data = data.float().unsqueeze(1)
                predictions = model(data)
                targets = targets.unsqueeze(1)
                loss = loss_function(predictions, targets)
                epoch_loss += loss+ 1/(torch.log(loss))
                #Backward
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            print("loss: ",epoch_loss/i)
                

In [165]:
TrainRCNN(model,train_loader,num_epochs,loss_function)


Epoch: 0
loss:  tensor(-1.3105, grad_fn=<DivBackward0>)
Epoch: 1
loss:  tensor(-0.3325, grad_fn=<DivBackward0>)
Epoch: 2
loss:  tensor(-0.3286, grad_fn=<DivBackward0>)
Epoch: 3
loss:  tensor(-0.3254, grad_fn=<DivBackward0>)
Epoch: 4
loss:  tensor(-0.3227, grad_fn=<DivBackward0>)
Epoch: 5
loss:  tensor(-0.3203, grad_fn=<DivBackward0>)
Epoch: 6
loss:  tensor(-0.3183, grad_fn=<DivBackward0>)
Epoch: 7
loss:  tensor(-0.3166, grad_fn=<DivBackward0>)
Epoch: 8
loss:  tensor(-0.3151, grad_fn=<DivBackward0>)
Epoch: 9
loss:  tensor(-0.3138, grad_fn=<DivBackward0>)


In [166]:
CheckAccuracy(model,x_train,preprocess.unscaled_y_train)

Got 837630 / 6410301 with accuracy 0.13067


In [167]:
CheckAccuracy(model,x_test,preprocess.unscaled_y_test)

Got 209408 / 1602616 with accuracy 0.13067


# Contractive autoencoder based model

In [209]:

class CAE(nn.Module):
    def __init__(self):
        super(CAE, self).__init__()

        self.enc1 = nn.Linear(166, 120) # Encoder
        self.enc2 = nn.Linear(120, 80) 
        self.enc3 = nn.Linear(80, 40) 

        self.dec1 = nn.Linear(40,80)
        self.dec2 = nn.Linear(80,120)
        self.dec3 = nn.Linear(120,166)
        self.output = nn.Linear(166,163)

        
        self.relu = nn.ReLU()


    def encoder(self, x):
        x = self.enc1(x)
        x = self.relu(x)
        x = self.enc2(x)
        x = self.relu(x)
        x = self.enc3(x)
        x = self.relu(x)
        return x

    def decoder(self,z):
        z = self.dec1(z)
        z = self.relu(z)
        z = self.dec2(z)
        z = self.relu(z)
        z = self.dec3(z)
        z = self.relu(z)
        z = self.output(z)
        return z

    def forward(self, x):
            h1 = self.encoder(x)
            h2 = self.decoder(h1)
            return h1, h2


In [210]:
model = CAE()
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr = learning_rate) 
batch_size = 30
num_epochs = 10

In [195]:
   def TrainCAE(train_loader,num_epochs):
        for epoch in range(num_epochs):
            acc=0
            print("Epoch: " + str(epoch))
            epoch_loss = 0
            for i, (data, targets) in enumerate(train_loader):
                output_enc,predictions = model(data)
                cae_loss = torch.norm(torch.autograd.functional.jacobian(model.encoder, data, create_graph=True))
                reconstruct_loss = loss_function(predictions,targets)
                loss = cae_loss + reconstruct_loss
                #Backward
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            print("loss: ",epoch_loss/i)
            
                

In [211]:
Ds = MyDataset(x_train, y_train)
train_loader=DataLoader(Ds,batch_size,shuffle=True)

In [213]:
TrainCAE(train_loader,num_epochs)

Epoch: 0
loss:  tensor(0.2252, grad_fn=<DivBackward0>)
Epoch: 1
loss:  tensor(0.1097, grad_fn=<DivBackward0>)
Epoch: 2
loss:  tensor(0.1095, grad_fn=<DivBackward0>)
Epoch: 3
loss:  tensor(0.1095, grad_fn=<DivBackward0>)
Epoch: 4
loss:  tensor(0.1094, grad_fn=<DivBackward0>)
Epoch: 5
loss:  tensor(0.1093, grad_fn=<DivBackward0>)
Epoch: 6
loss:  tensor(0.1093, grad_fn=<DivBackward0>)
Epoch: 7
loss:  tensor(0.1092, grad_fn=<DivBackward0>)
Epoch: 8
loss:  tensor(0.1092, grad_fn=<DivBackward0>)
Epoch: 9
loss:  tensor(0.1092, grad_fn=<DivBackward0>)


In [225]:
CheckAccuracy(model,x_train,preprocess.unscaled_y_train)

Got 250450 / 6410301 with accuracy 0.03907


In [226]:
CheckAccuracy(model,x_test,preprocess.unscaled_y_test)

Got 59474 / 1602616 with accuracy 0.03711


# CNN based autoencoder model suggested on paper

In [217]:
#Convolution 1D
class CnnEncDec(nn.Module): 
    def __init__(self):  
        super(CnnEncDec, self).__init__()

        self.enc1 = torch.nn.Conv1d(in_channels=1, out_channels=30, kernel_size=10, stride=2,
                        padding=0, dilation=1, groups=1, bias=True, 
                        padding_mode='zeros', device=None, dtype=None)
        
        self.relu = nn.ReLU()
        self.enc2 = nn.Conv1d(in_channels=30, out_channels=15, kernel_size=10, stride=5,
                        padding=0, dilation=1, groups=1, bias=True, 
                        padding_mode='zeros', device=None, dtype=None)
        
        self.enc3 = nn.Conv1d(in_channels=15, out_channels=8, kernel_size=10, stride=2,
                        padding=0, dilation=1, groups=1, bias=True, 
                        padding_mode='zeros', device=None, dtype=None)

        
        self.softmax = nn.Softmax()
        self.dense1 = nn.Linear(3,100)
        self.dense2 = nn.Linear(100,163)
        
        self.dec1 = torch.nn.ConvTranspose1d(in_channels=8, out_channels=15, kernel_size=10, stride=2, padding=0, 
                                             output_padding=0, groups=1, bias=True, dilation=1, padding_mode='zeros')
            
        self.dec2 = torch.nn.ConvTranspose1d(in_channels=15, out_channels=25, kernel_size=10, stride=5, padding=0, 
                                             output_padding=0, groups=1, bias=True, dilation=1, padding_mode='zeros')
        
        self.dec3 = torch.nn.ConvTranspose1d(in_channels=25, out_channels=30, kernel_size=10, stride=2, padding=0, 
                                             output_padding=0, groups=1, bias=True, dilation=1, padding_mode='zeros')
        
        self.dec4 = torch.nn.ConvTranspose1d(in_channels=30, out_channels=1, kernel_size=10, stride=1, padding=0, 
                                             output_padding=0, groups=1, bias=True, dilation=1, padding_mode='zeros')
        self.dense3 = nn.Linear(3367,2000)
        self.dense4 = nn.Linear(2000,1000)
        self.dense5 =  nn.Linear(1000,163)
                      
            
    def forward(self,x):
        x= self.enc1(x)
        x = self.relu(x)
        x = self.enc2(x)
        x= self.relu(x)
        x = self.enc3(x)
        x = self.relu(x)
        x = self.dense1(x)
        x = self.relu(x)
        x = self.dense2(x)
        x = self.relu(x)
        x = self.dec1(x)
        x = self.relu(x)
        x = self.dec2(x)
        x = self.relu(x)
        x = self.dec3(x)
        x = self.relu(x)
        x = self.dec4(x)
        x = self.relu(x)
        x = self.dense3(x)
        x = self.relu(x)
        x = self.dense4(x) 
        x = self.relu(x)
        x = self.dense5(x)
        return x


# loss function = 10*log(mse)

In [56]:
   def TrainCNN(train_loader,num_epochs,loss_function):
        for epoch in range(num_epochs):
            acc=0
            print("Epoch: " + str(epoch))
            epoch_loss = 0
            for i, (data, targets) in enumerate(train_loader):
                data = data.float().unsqueeze(1)
                predictions = model(data)
                targets = targets.unsqueeze(1)
                loss = loss_function(predictions, targets)
                epoch_loss += 10*torch.log(loss)
                #Backward
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            print("loss: ",epoch_loss/i)
                

In [ ]:
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr = learning_rate) 
batch_size = 30
num_epochs = 10

In [57]:
TrainCNN(train_loader,num_epochs,loss_function)


Epoch: 0
loss:  tensor(-25.7381, grad_fn=<DivBackward0>)
Epoch: 1
loss:  tensor(-25.9350, grad_fn=<DivBackward0>)
Epoch: 2
loss:  tensor(-26.0275, grad_fn=<DivBackward0>)
Epoch: 3
loss:  tensor(-26.0517, grad_fn=<DivBackward0>)
Epoch: 4
loss:  tensor(-26.0597, grad_fn=<DivBackward0>)
Epoch: 5
loss:  tensor(-26.0724, grad_fn=<DivBackward0>)
Epoch: 6
loss:  tensor(-26.0833, grad_fn=<DivBackward0>)
Epoch: 7
loss:  tensor(-26.0978, grad_fn=<DivBackward0>)
Epoch: 8
loss:  tensor(-26.1038, grad_fn=<DivBackward0>)
Epoch: 9
loss:  tensor(-26.1161, grad_fn=<DivBackward0>)


In [58]:
CheckAccuracy(model,x_train,preprocess.unscaled_y_train)

Got 2694718 / 6410301 with accuracy 0.42037


In [227]:
CheckAccuracy(model,x_test,preprocess.unscaled_y_test)

Got 250450 / 6410301 with accuracy 0.03907
